In [1]:
import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

Using TensorFlow backend.
/home/yt/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/yt/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/yt/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/yt/anaconda3/envs/tf_gpu/l

In [2]:
train = pd.read_csv('Cowen/train_new.csv')
train.head()

,image,class
0,0027.mp4_frame72.jpg,Amusement
1,0032.mp4_frame190.jpg,Amusement
2,0032.mp4_frame397.jpg,Amusement
3,0042.mp4_frame64.jpg,Aesthetic Appreciation
4,0053.mp4_frame729.jpg,Disgust


In [3]:
train.shape[0]

11646

In [5]:
# creating an empty list
train_image = []
X=[]
# for loop to read and store frames
#for i in tqdm(range(train.shape[0])):
#for i in tqdm(range(8000)):  # memory constraint
for i in tqdm(range(4000)):  # memory constraint

    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img('train_1/'+train['image'][i], target_size=(224,224,3))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255
    # appending the image to the train_image list
    train_image.append(img)
    
# converting the list to numpy array
X = np.array(train_image)

# shape of the array
X.shape

100%|██████████| 4000/4000 [00:23<00:00, 172.00it/s]


(4000, 224, 224, 3)

In [6]:
train_y =pd.read_csv('Cowen/train_y.csv')
train_y.head()

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,0.0,0.0,0.0,0.58333,0.0,0.0,0.0,0.16667,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,0.0,0.25,0.41667
1,0.3,0.0,0.1,0.40000,0.0,0.0,0.3,0.10000,0.1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.2,0.0,0.10,0.00000
2,0.3,0.0,0.1,0.40000,0.0,0.0,0.3,0.10000,0.1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.2,0.0,0.10,0.00000
3,0.0,0.0,0.4,0.10000,0.0,0.0,0.2,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.10,0.00000
4,0.0,0.0,0.0,0.30000,0.0,0.0,0.0,0.10000,0.0,0.0,...,0.1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.10,0.00000


In [8]:
# separating the target
#y = train[:8000]['class']
y = train_y[:4000].values

y.shape

(4000, 33)

In [9]:

# creating the training and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)

In [10]:
# creating dummies of target variable for train and validation set
#y_train = pd.get_dummies(y_train)
#y_test = pd.get_dummies(y_test)

In [11]:
y_test.shape

(800, 33)

In [12]:
# creating the base model of pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False)

In [13]:
# extracting features for training frames
X_train = base_model.predict(X_train)
X_train.shape

(3200, 7, 7, 512)

In [14]:
# extracting features for validation frames
X_test = base_model.predict(X_test)
X_test.shape

(800, 7, 7, 512)

In [15]:
# reshaping the training as well as validation frames in single dimension
X_train = X_train.reshape(3200, 7*7*512)
X_test = X_test.reshape(800, 7*7*512)

In [16]:
# normalizing the pixel values
max = X_train.max()
X_train = X_train/max
X_test = X_test/max

In [17]:
# shape of images
X_train.shape

(3200, 25088)

In [18]:
#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(25088,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
#model.add(Dense(101, activation='softmax'))
# We have 21 classes 
model.add(Dense(33, activation='sigmoid'))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [19]:
# defining a function to save the weights of best model
from keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [20]:
# compiling the model
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [21]:
# training the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=128)

Train on 3200 samples, validate on 800 samples
Epoch 1/10
3200/3200 [==============================] - 8s 3ms/step - loss: 0.3878 - acc: 0.6613 - val_loss: 0.1930 - val_acc: 0.7600
Epoch 2/10
3200/3200 [==============================] - 6s 2ms/step - loss: 0.2308 - acc: 0.7513 - val_loss: 0.1783 - val_acc: 0.7600
Epoch 3/10
3200/3200 [==============================] - 6s 2ms/step - loss: 0.2091 - acc: 0.7567 - val_loss: 0.1833 - val_acc: 0.7600
Epoch 4/10
3200/3200 [==============================] - 6s 2ms/step - loss: 0.1960 - acc: 0.7579 - val_loss: 0.1647 - val_acc: 0.7600
Epoch 5/10
3200/3200 [==============================] - 6s 2ms/step - loss: 0.1847 - acc: 0.7585 - val_loss: 0.1617 - val_acc: 0.7600
Epoch 6/10
3200/3200 [==============================] - 6s 2ms/step - loss: 0.1765 - acc: 0.7587 - val_loss: 0.1555 - val_acc: 0.7600
Epoch 7/10
3200/3200 [==============================] - 6s 2ms/step - loss: 0.1711 - acc: 0.7589 - val_loss: 0.1511 - val_acc: 0.7600
Epoch 8/10
3200